# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [60]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [61]:
vac = pd.read_csv("../output_data/cities.csv")
vac

,city,lat,long,temp,hum,cld,wind,country
0,umm lajj,25.02,37.27,75.00,48,0,6.33,SA
1,vaini,-21.20,-175.20,80.60,74,20,10.29,TO
2,victoria,22.29,114.16,77.65,74,68,4.00,HK
3,oranjemund,-28.55,16.43,61.95,62,27,3.62,NaN
4,punta arenas,-53.15,-70.92,48.00,76,100,17.00,CL
...,...,...,...,...,...,...,...,...
563,sechura,-5.56,-80.82,76.71,64,21,14.92,PE
564,kangaatsiaq,68.31,-53.46,32.05,98,100,16.37,GL
565,kasempa,-13.46,25.83,62.37,56,11,3.65,ZM
566,novo aripuana,-5.12,-60.38,76.10,88,56,1.48,BR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [62]:
gmaps.configure(api_key=g_key)

In [63]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [64]:
locations = vac[['lat', 'long']]
humidity = vac['hum']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [65]:
vac

,city,lat,long,temp,hum,cld,wind,country
0,umm lajj,25.02,37.27,75.00,48,0,6.33,SA
1,vaini,-21.20,-175.20,80.60,74,20,10.29,TO
2,victoria,22.29,114.16,77.65,74,68,4.00,HK
3,oranjemund,-28.55,16.43,61.95,62,27,3.62,NaN
4,punta arenas,-53.15,-70.92,48.00,76,100,17.00,CL
...,...,...,...,...,...,...,...,...
563,sechura,-5.56,-80.82,76.71,64,21,14.92,PE
564,kangaatsiaq,68.31,-53.46,32.05,98,100,16.37,GL
565,kasempa,-13.46,25.83,62.37,56,11,3.65,ZM
566,novo aripuana,-5.12,-60.38,76.10,88,56,1.48,BR


In [66]:
# Temp 50 and 60 
# Wind 5 - 15 
# Zero cloudiness
# filter or locate 
mask = (vac.temp > 50) & (vac.temp < 60) & (vac.wind > 0) & (vac.wind < 15) & (vac.cld == 0)
vac_sub = vac.loc[mask]
vac_sub

,city,lat,long,temp,hum,cld,wind,country
80,san rafael,-34.62,-68.33,59.68,44,0,5.14,AR
113,xining,36.62,101.77,58.82,13,0,2.08,CN
115,yulara,-25.24,130.99,51.80,53,0,10.29,AU
171,manzil salim,35.86,8.48,57.20,71,0,2.24,TN
189,valparaiso,-33.04,-71.63,57.20,87,0,3.36,CL
218,mvuma,-19.28,30.53,58.08,51,0,7.74,ZW
247,alice springs,-23.70,133.88,55.40,47,0,9.17,AU
260,carnarvon,-24.87,113.63,57.20,54,0,4.70,AU
348,gat,31.61,34.76,59.99,65,0,2.24,IL
374,tazmalt,36.39,4.40,59.00,93,0,4.70,DZ


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [67]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "key": g_key}
for index, row in vac_sub.iterrows():

    # get lat long from df
    city = f"{row['lat']} , {row['long']}"

    # add keyword to params dict
    params["location"] = city

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        
        vac_sub.loc[index, 'Hotel Name'] = results[0]['name']
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 80: -34.62 , -68.33.
Closest -34.62 , -68.33 hotel is Hotel Residencial Independencia.
------------
Retrieving Results for Index 113: 36.62 , 101.77.


C:\Users\ckrokus\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\ckrokus\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest 36.62 , 101.77 hotel is Sports High School Family Dependents' Courtyard 2 Building.
------------
Retrieving Results for Index 115: -25.24 , 130.99.
Closest -25.24 , 130.99 hotel is Ayers Rock Campground.
------------
Retrieving Results for Index 171: 35.86 , 8.48.
Closest 35.86 , 8.48 hotel is mohamed ben khlil omri.
------------
Retrieving Results for Index 189: -33.04 , -71.63.
Closest -33.04 , -71.63 hotel is Amazing Hostel Cerro Alegre.
------------
Retrieving Results for Index 218: -19.28 , 30.53.
Missing field/result... skipping.
------------
Retrieving Results for Index 247: -23.7 , 133.88.
Closest -23.7 , 133.88 hotel is The Diplomat Motel.
------------
Retrieving Results for Index 260: -24.87 , 113.63.
Closest -24.87 , 113.63 hotel is Tilly Gia home sweet home.
------------
Retrieving Results for Index 348: 31.61 , 34.76.
Closest 31.61 , 34.76 hotel is שדרות גת 88 קרית גת.
------------
Retrieving Results for Index 374: 36.39 , 4.4.
Closest 36.39 , 4.4 hotel is Hôtel Nu

In [68]:
vac_sub

,city,lat,long,temp,hum,cld,wind,country,Hotel Name
80,san rafael,-34.62,-68.33,59.68,44,0,5.14,AR,Hotel Residencial Independencia
113,xining,36.62,101.77,58.82,13,0,2.08,CN,Sports High School Family Dependents' Courtyar...
115,yulara,-25.24,130.99,51.80,53,0,10.29,AU,Ayers Rock Campground
171,manzil salim,35.86,8.48,57.20,71,0,2.24,TN,mohamed ben khlil omri
189,valparaiso,-33.04,-71.63,57.20,87,0,3.36,CL,Amazing Hostel Cerro Alegre
218,mvuma,-19.28,30.53,58.08,51,0,7.74,ZW,NaN
247,alice springs,-23.70,133.88,55.40,47,0,9.17,AU,The Diplomat Motel
260,carnarvon,-24.87,113.63,57.20,54,0,4.70,AU,Tilly Gia home sweet home
348,gat,31.61,34.76,59.99,65,0,2.24,IL,שדרות גת 88 קרית גת
374,tazmalt,36.39,4.40,59.00,93,0,4.70,DZ,Hôtel Numédia


In [73]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vac_sub.iterrows()]
locations = vac_sub[["lat", "long"]]

In [74]:
# Add marker layer ontop of heat map
locations = vac_sub[['lat', 'long']]
fig.add_layer(gmaps.marker_layer(locations, info_box_content = hotel_info))
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))